# Random intervals

In [1]:
import numpy as np
import pandas as pd
import random
import datetime as dt
import yfinance as yf
import plotly.express as px

# Función Random Intervals

Esta función recibe un dataframe con la columna Close y crea "subconjuntos" de datos que contienen un tamaño aleatorio, estos subconjuntos de datos pueden intersectarse o no, pueden tomar los límites de los datos o no, realmente depende de la aleatoriedad del algoritmo.

La función devuelve dos objetos, el primero es un dataframe "diseñado" para graficar todos los subconjuntos de datos creados y el segundo es una lista con los subconjuntos de datos creados.

Los parametros son: 
* df, el dataframe que queremos "segmentar".
* n, el número de subconjuntos deseado.
* min_wt, el tamaño mínimo de los subconjuntos. Default es 14.
* max_wt, el tamaño máximo de los subconjunots. Default es 128.

In [2]:
def random_intervals(df:pd.DataFrame, n:int, min_wt=14, max_wt=128):
    assert type(df.index) == pd.core.indexes.datetimes.DatetimeIndex, "Data Index must be pandas DatetimeIndex"

    centroides = np.sort(random.sample(range(1, len(df.index)-2), n))
    epsilons = np.random.randint(min_wt, max_wt, n)
    l_inf = np.clip(centroides - epsilons, 0, len(df.index)-2)
    l_sup = np.clip(centroides + epsilons, 1, len(df.index)-1)

    # Getting the date pairs
    pairs = np.transpose([df.index[l_inf], df.index[l_sup]])

    # Segmenting the dataframe using the pairs of dates that were choosen randomly
    if len(df.columns) == 1:
        batches = []
        for pair in pairs:
            batch = df[(df.index>=pair[0]) & (df.index<=pair[1])]
            batch.columns = [f"Close from {str(pair[0])[:10]} to {str(pair[1])[:10]}"]
            batches.append(batch)

        to_plot = df.copy()
        for b in batches:
            to_plot = pd.merge(to_plot, b, left_index=True, right_index=True, how="left")

    else:
        batches = []
        for pair in pairs:
            batch = df[(df.index>=pair[0]) & (df.index<=pair[1])]
            columns = [str(t) + f" Close from {str(pair[0])[:10]} to {str(pair[1])[:10]}" for t in batch.columns]
            batch.columns = columns
            batches.append(batch)
        
        to_plot = df.copy()
        for b in batches:
            to_plot = pd.merge(to_plot, b, left_index=True, right_index=True, how="left")
   
    return  to_plot, batches

# Ejemplo

Descargamos el precio "Close" de TSLA para el último año.

In [3]:
df = yf.download("TSLA", start="2022-01-01")[["Close"]]
df.head()

[*********************100%***********************]  1 of 1 completed


,Close
Date,
2022-01-03,1199.780029
2022-01-04,1149.589966
2022-01-05,1088.119995
2022-01-06,1064.699951
2022-01-07,1026.959961


Guardamos los dos objetos que arroja la función, el primero será para graficar todos los suconjuntos, el segundo es la lista que contiene los subconjuntos.


In [4]:
to_plot, batches = random_intervals(df=df, n=10)

Ploteamos el objeto destinado a ser ploteado

In [5]:
px.line(to_plot)

In [6]:
batches[-1]

,Close from 2022-03-17 to 2022-07-25
Date,
2022-03-17,871.599976
2022-03-18,905.390015
2022-03-21,921.159973
2022-03-22,993.979980
2022-03-23,999.109985
...,...
2022-07-19,736.590027
2022-07-20,742.500000
2022-07-21,815.119995


In [7]:
df = yf.download(["TSLA", "GOOG"], start="2022-01-01")["Close"]
df.head()

[*********************100%***********************]  2 of 2 completed


,GOOG,TSLA
Date,,
2022-01-03,145.074493,1199.780029
2022-01-04,144.416504,1149.589966
2022-01-05,137.653503,1088.119995
2022-01-06,137.550995,1064.699951
2022-01-07,137.004501,1026.959961


In [8]:
to_plot, batches = random_intervals(df=df, n=10)

In [9]:
px.line(to_plot)

# Random not overlapping intervals

Esta función recibe un dataframe con la columna Close y crea "subconjuntos" de datos que contienen un tamaño aleatorio, estos subconjuntos no se traslapan y siempre toman los limites del dataframe de donde se originan.

La función devuelve dos objetos, el primero es un dataframe "diseñado" para graficar todos los subconjuntos de datos creados y el segundo es una lista con los subconjuntos de datos creados.

Los parametros son: 
* df, el dataframe que queremos "segmentar".
* n, el número de subconjuntos deseado.

In [14]:
def random_not_overlapping_intervals(df:pd.DataFrame, n:int, min_wt=14):
    assert type(df.index) == pd.core.indexes.datetimes.DatetimeIndex, "Data Index must be pandas DatetimeIndex"
    assert (len(df.index) // min_wt) > n, f"The length of the data is insufficient for the minimum size for each batch, try using max n = {int(len(df.index) // min_wt) - 1}"
    
    intervals=np.sort(random.sample(range(min_wt-1, len(df.index)-(2+min_wt), min_wt-1), n-1))
    
    # Getting the date pairs
    pairs = np.transpose([df.index[[0] + list(np.sort(intervals))], df.index[list(np.sort(intervals)) + [-1]]])

    # Segmenting the dataframe using the pairs of dates that were choosen randomly
    if len(df.columns) == 1:
        batches = []
        for pair in pairs:
            batch = df[(df.index >= pair[0]) & (df.index <= pair[1])]
            batch.columns = [f"Close from {str(pair[0])[:10]} to {str(pair[1])[:10]}"]
            batches.append(batch)
        
        to_plot = df.copy()
        for b in batches:
            to_plot = pd.merge(to_plot, b, left_index=True, right_index=True, how="left")
    else:
        batches = []
        for pair in pairs:
            batch = df[(df.index>=pair[0]) & (df.index<=pair[1])]
            columns = [str(t) + f" Close from {str(pair[0])[:10]} to {str(pair[1])[:10]}" for t in batch.columns]
            batch.columns = columns
            batches.append(batch)
        
        to_plot = df.copy()
        for b in batches:
            to_plot = pd.merge(to_plot, b, left_index=True, right_index=True, how="left")
    return to_plot, batches

# Example

Guardamos los dos objetos que arroja la función, el primero será para graficar todos los subconjuntos, el segundo es la lista que contiene los subconjuntos.

In [15]:
len(df)

140

In [16]:
to_plot, batches = random_not_overlapping_intervals(df=df, n=8)

Ploteamos el objeto destinado a ser ploteado

In [17]:
px.line(to_plot)

In [18]:
df.index.min(), df.index.max()

(Timestamp('2022-01-03 00:00:00'), Timestamp('2022-07-25 00:00:00'))

In [19]:
batches[0].index.min(), batches[-1].index.max()

(Timestamp('2022-01-03 00:00:00'), Timestamp('2022-07-25 00:00:00'))

In [20]:
for a,b in zip(range(len(batches)), batches):
    print(a,b.shape[0])


0 14
1 14
2 14
3 27
4 14
5 14
6 27
7 23


# Example 2

In [21]:
df = yf.download(["TSLA", "GOOG", "NVDA"], start="2022-01-01")["Close"]
df.head()

[*********************100%***********************]  3 of 3 completed


,GOOG,NVDA,TSLA
Date,,,
2022-01-03,145.074493,301.209991,1199.780029
2022-01-04,144.416504,292.899994,1149.589966
2022-01-05,137.653503,276.040009,1088.119995
2022-01-06,137.550995,281.779999,1064.699951
2022-01-07,137.004501,272.470001,1026.959961


In [22]:
to_plot, batches = random_not_overlapping_intervals(df=df, n=8)

In [23]:
px.line(to_plot)